Liveness-, Readiness- und Startup-Tests
=================================

Liveness probes help to know when a container needs to be restarted. For example, liveness probes could catch a deadlock when an application is running but can't make any progress (eg current database does not start).

The example below launches a pod which crashes after 15 seconds because the monitored file is no longer present:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-exec
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/busybox
    imagePullPolicy: IfNotPresent
    args:
    - /bin/sh
    - -c
    - touch /tmp/healthy; sleep 30; rm -rf /tmp/healthy; sleep 600
    livenessProbe:
      exec:
        command:
        - cat
        - /tmp/healthy
      initialDelaySeconds: 5
      periodSeconds: 5
%EOF%

We can check this by running several times:

In [ ]:
! kubectl describe pod liveness-exec

Also, when displaying the pod, the multiple restarts should be visible:

In [ ]:
! kubectl get pods liveness-exec

Cleanup

In [ ]:
! kubectl delete pod/liveness-exec

***
### Liveness-Tests HTTP Variante

The same also works with HTTP ports:


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-http
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/liveness
    imagePullPolicy: IfNotPresent    
    args:
    - /server
    livenessProbe:
      httpGet:
        path: /healthz
        port: 8080
        httpHeaders:
        - name: Custom-Header
          value: Awesome
      initialDelaySeconds: 3
      periodSeconds: 3
%EOF%


The program in the pod is written to return an HTTP 200 response for the first 10 seconds and then an HTTP 500 response.

    http.HandleFunc("/healthz", func(w http.ResponseWriter, r *http.Request) {
      duration := time.Now().Sub(started)
      if duration.Seconds() > 10 {
        w.WriteHeader(500)
        w.Write([]byte(fmt.Sprintf("error: %v", duration.Seconds())))
      } else {
        w.WriteHeader(200)
        w.Write([]byte("ok"))
      }
    })



In [ ]:
! kubectl describe pod liveness-http
! kubectl get pods liveness-http

Clean up

In [ ]:
! kubectl delete pods liveness-http

***
Startup-Tests
-------------------

Startup probes help determine when a container application has started. If such a probe is configured, the aliveness and readiness checks are disabled until it succeeds to ensure that these probes do not interfere with application startup ören.

Let's take a legacy application that simply takes a certain amount of time to start. A liveness test could result in the application being killed before it can even get into a `ready` state.

The problem can be worked around with `startupProbe`. This ensures that `livenessProbe` is only executed after `failureThreshold * periodSeconds`.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-http
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/liveness
    imagePullPolicy: IfNotPresent    
    args:
    - /server
    
    ports:
    - name: liveness-port
      containerPort: 8080
      hostPort: 8080
    
    livenessProbe:
     httpGet:
       path: /healthz
       port: liveness-port
     failureThreshold: 1
     periodSeconds: 10

    startupProbe:
     httpGet:
       path: /healthz
       port: liveness-port
     failureThreshold: 30
     periodSeconds: 10
%EOF%

In [ ]:
! kubectl describe pod liveness-http
! kubectl get pods liveness-http

Clean up

In [ ]:
! kubectl delete pods liveness-http

***
Readiness-Tests
---------------

Readiness probes help determine when a container is ready to accept traffic. A pod is considered ready when all of its containers are ready. If a pod is not ready, it is removed from the Service Load balancers removed.

Sometimes applications can temporarily not serve the data traffic, e.g. because the application has to process a large amount of data.

Kubernetes should detect this situation and not start the application, but make sure that it doesn't receive any new data.

There are readiness tests for this, which separate the application from the service and thus ensure that no data is received.


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-exec
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/busybox
    imagePullPolicy: IfNotPresent
    args:
    - /bin/sh
    - -c
    - touch /tmp/healthy; sleep 30; rm -rf /tmp/healthy; sleep 600
    readinessProbe:
      exec:
        command:
        - cat
        - /tmp/healthy
      initialDelaySeconds: 5
      periodSeconds: 5
%EOF%

In [ ]:
! kubectl describe pod liveness-exec
! kubectl get pods/liveness-exec

Clean up

In [ ]:
! kubectl delete pod/liveness-exec